In [ ]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

In [ ]:
class_name = "Work"
count = CLIENT.query.aggregate(class_name).with_meta_count().do()
jprint(count)

In [ ]:
jprint(
    CLIENT.query
        .get(class_name="Work", properties=[])
        .with_limit(20)
        .with_near_text({"concepts": ["pictures of musicians"]})
        .with_group_by(["collection"], groups=10, objects_per_group=2)
        .with_additional([
            """
            group {
                id
                groupedBy {
                    value
                }
                count
                hits {
                    identifier
                    title
                    subject
                }
            }
            """
        ])
        .do()
)

In [ ]:
work=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title", "collection"])
        .with_limit(1)
        .with_additional(["vector"])
        .do()
)["data"]["Get"]["Work"][0]
vector=work["_additional"]["vector"]
jprint(work, expanded=False)

In [ ]:
result=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title"])
        .with_near_vector({"vector": vector})
        .with_limit(100)
        .do()
)
jprint(result)

In [ ]:
result=(
    CLIENT.query
        .get(class_name="Work", properties=["collection", "identifier", "title"])
        .with_limit(100)
        .with_where({
            "path": ["collection"],
            "operator": "NotEqual",
            "valueText": work["collection"]
        })
        .with_near_vector({"vector": vector})
        .do()
)
jprint(result)